In [44]:
import pandas as pd
from pathlib import Path
import os
import joblib
import numpy as np
from mapie.metrics import (
    classification_coverage_score,
    classification_mean_width_score
)
import re

In [36]:
from utils.model_production_data_processing_utils import cluster_with_min_size

c:\Users\Anton CONRAD\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
root = Path(os.getcwd()).parent

In [4]:
obj = joblib.load(root / "models" / "models_clustering_24.joblib")

In [66]:
obj2 = joblib.load(root / "models" / "models_clustering_SPCI_ng_24.joblib")

In [67]:
type(obj2)

dict

In [68]:
obj2.keys()

dict_keys([('RF', 1, 'vanilla'), ('RF', 1, 'mondrian'), ('RF', 2, 'vanilla'), ('RF', 2, 'mondrian'), ('RF', 3, 'vanilla'), ('RF', 3, 'mondrian'), ('RF', 4, 'vanilla'), ('RF', 4, 'mondrian'), ('RF', 5, 'vanilla'), ('RF', 5, 'mondrian'), ('RF', 6, 'vanilla'), ('RF', 6, 'mondrian'), ('RF', 7, 'vanilla'), ('RF', 7, 'mondrian'), ('RF', 8, 'vanilla'), ('RF', 8, 'mondrian'), ('RF', 9, 'vanilla'), ('RF', 9, 'mondrian'), ('RF', 10, 'vanilla'), ('RF', 10, 'mondrian'), ('RF', 11, 'vanilla'), ('RF', 11, 'mondrian'), ('RF', 12, 'vanilla'), ('RF', 12, 'mondrian'), ('RF', 13, 'vanilla'), ('RF', 13, 'mondrian'), ('RF', 14, 'vanilla'), ('RF', 14, 'mondrian'), ('RF', 15, 'vanilla'), ('RF', 15, 'mondrian'), ('RF', 16, 'vanilla'), ('RF', 16, 'mondrian'), ('RF', 17, 'vanilla'), ('RF', 17, 'mondrian'), ('RF', 18, 'vanilla'), ('RF', 18, 'mondrian'), ('RF', 19, 'vanilla'), ('RF', 19, 'mondrian'), ('RF', 20, 'vanilla'), ('RF', 20, 'mondrian'), ('RF', 21, 'vanilla'), ('RF', 21, 'mondrian'), ('RF', 22, 'vanilla'

In [47]:
df = pd.read_csv(root / "data/DATA.csv")
nb_nan_par_ligne = df.isna().sum(axis=1)

df = df[nb_nan_par_ligne < 495]

In [48]:
def build_X_s(df_sub: pd.DataFrame, prefixes: list, static_cols: list, n: int) -> np.ndarray:
    # on garde student_id + les n premiers items
    dyn_cols = [
    col for col in df_sub.columns
    if any(col.startswith(pref) for pref in prefixes[:n])
    ]
    keep = ["email"] + static_cols + dyn_cols
    return df_sub[keep].set_index("email").values

In [49]:
mark_cols = [c for c in df.columns if c.endswith("mark")]
prefixes = list(dict.fromkeys(c.rsplit("_",1)[0] for c in mark_cols))
static_cols = []

In [75]:
X = build_X_s(df.fillna(0), prefixes, static_cols, 3)

In [76]:
dfcpool = df[[c for c in df.columns if c.startswith("B-CPE-100")]]
pat = re.compile(r"B-CPE-100_cpoolday\d+_\d{2} - task\d+_passed")
cols_keep = [c for c in dfcpool.columns if not pat.match(c)]
dfcpool_mark = dfcpool[cols_keep]
X_pool = dfcpool_mark.fillna(0)

In [77]:
df2, info = cluster_with_min_size(
    df, X_pool, n_clusters=4, min_cluster_size=50, random_state=42)

DF plain + clustering done
nombre d'élèves par cluster :
0    117
1    490
2     30
3    213
Name: count, dtype: int64
Clusters trop petits à réaffecter : [2]
Nouvelles tailles de clusters :
0    117
1    490
3    243
Name: count, dtype: int64


In [90]:
res = []
for n in range(1, 16):
    mod = obj2[('GB', n, 'vanilla')]
    X = build_X_s(df2.fillna(0), prefixes, static_cols, n)
    yp_van, yps_van = mod.predict(X, alpha=0.1) # partition=df2['clusters'])
    pset_van = yps_van[:, :, 0]
    print(classification_mean_width_score(pset_van))
    res.append(classification_mean_width_score(pset_van))
print("moy", np.mean(res))

1.391764705882353
1.4823529411764707
1.3541176470588234
1.3741176470588234
1.3858823529411766
1.596470588235294
1.6258823529411766
1.5294117647058822
1.531764705882353
1.5541176470588236
1.5411764705882354
1.591764705882353
1.3952941176470588
1.3976470588235295
1.3529411764705883
moy 1.4736470588235295


In [ ]:
df.head()

In [ ]:
col_series = df.drop(columns=['email']).columns.to_series()
suffixes = col_series.apply(lambda x: x.split("_")[1])
ordered_suffixes = suffixes.unique()
# 2) Groupement des colonnes par suffixe
dfs = {}
for suffix in ordered_suffixes:
    cols_for_suffix = [c for c in col_series if c.split("_")[1] == suffix]
    subdf = df[cols_for_suffix].copy()
    dfs[suffix] = subdf
    if True:
        print(f"Suffixe = {suffix} → shape {subdf.shape}")


In [ ]:
n = 20  # nombre de colonnes à afficher
print(df.isna().sum().sort_values(ascending=False).head(n))
